1. Label data using Named Entity Recognition
2. Split data into train and test
3. train model
4. test model

In [37]:
import pandas as pd

import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span, DocBin

from scholarly import scholarly

import requests
import mimetypes
import os

import uuid

import PyPDF2

from PyPDF2 import PdfReader
import concurrent.futures

from spacy import displacy
import random

In [2]:
ROOT_FOLDER = '../../../'

In [3]:
DATA_FOLDER = '../../../data/'

In [4]:
DETECT_ML_MODEL_FILES_FOLDER = '../../../detect_ml_model_files/'

In [5]:
data_df = pd.read_parquet(f'{DATA_FOLDER}detect_ml_models.parquet')

In [6]:
# drop records where the file_name is Null
data_df = data_df.dropna()

In [7]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path
0,Linear regression,Linear regression plays a fundamental role in ...,2012,https://www.cs.columbia.edu/~djhsu/coms4771-f2...,"[wcgZZP8AAAAJ, , El-UNYoAAAAJ]",Linear Regression,7cedd2b0-ac03-49fb-9ef1-398b1a5d7fe1.pdf,../../../detect_ml_model_files/7cedd2b0-ac03-4...
1,Introduction to linear regression analysis,Equation (1.2) is called a linear regression m...,2021,http://sutlib2.sut.ac.th/sut_contents/H133678.pdf,"[5PboKNAAAAAJ, , y2eb6cQAAAAJ]",Linear Regression,91374318-7135-4f51-a3bc-720ececd88de.pdf,../../../detect_ml_model_files/91374318-7135-4...
2,Linear regression,"linear regression, a very simple approach for ...",2023,https://datamineaz.org/readings/ISL_chp3.pdf,"[KUIjZqgAAAAJ, bHZf-c8AAAAJ, tQVe-fAAAAAJ, ZpG...",Linear Regression,6d3627b9-6b0b-423c-8898-3185837a7617.pdf,../../../detect_ml_model_files/6d3627b9-6b0b-4...
4,Applied linear regression,Applied linear regression Applied linear regre...,2005,https://www.stat.cmu.edu/~brian/valerie/617-20...,[wlu6jZQAAAAJ],Linear Regression,c7fed73c-8e13-4770-a589-67708aab0b7e.pdf,../../../detect_ml_model_files/c7fed73c-8e13-4...
8,A review on linear regression comprehensive in...,We discuss linear regression and polynomial re...,2020,https://jastt.org/index.php/jasttpath/article/...,"[G9U01kwAAAAJ, aBdgHxkAAAAJ]",Linear Regression,f8d408d7-9c86-4d72-b6af-6538ed46970f.pdf,../../../detect_ml_model_files/f8d408d7-9c86-4...
...,...,...,...,...,...,...,...,...
1227,BERT: a review of applications in natural lang...,"relative to the original BERT model, which is ...",2021,https://arxiv.org/pdf/2103.11943,[],BERT,91ebf298-65bd-4429-8c60-b17cb80306fe.pdf,../../../detect_ml_model_files/91ebf298-65bd-4...
1228,What does BERT learn about the structure of la...,language structure learned by BERT. We first s...,2019,https://inria.hal.science/hal-02131630/document,"[X7SMP1EAAAAJ, HXUT9ZkAAAAJ, P7EtARsAAAAJ]",BERT,65f04db1-ad62-4f48-9e80-3ae5a11bf147.pdf,../../../detect_ml_model_files/65f04db1-ad62-4...
1229,Visualizing and understanding the effectivenes...,trajectories of fine-tuning BERT on specific d...,2019,https://arxiv.org/pdf/1908.05620,"[cqOLO7IAAAAJ, wEfQgPgAAAAJ, G-V1VpwAAAAJ, ]",BERT,03d43387-8413-413c-8ce8-92b3512cedfb.pdf,../../../detect_ml_model_files/03d43387-8413-4...
1230,A comprehensive survey on pretrained foundatio...,Pretrained Foundation Models (PFMs) are regard...,2024,https://arxiv.org/pdf/2302.09419,"[HWx73DcAAAAJ, AHg-JGIAAAAJ, nPvWFpkAAAAJ, fh1...",BERT,a60911d8-b83b-4c2a-8555-2ac3b09b5025.pdf,../../../detect_ml_model_files/a60911d8-b83b-4...


The query column represents the machine learning model name that was used to search Google Scholar.

There should be five papers per model.

Add a new column to represent an identifier/order for the paper within each query

In [8]:
#data_df['counter'] = data_df.groupby('query').cumcount() + 1
data_df.loc[:, 'counter'] = data_df.groupby('query').cumcount() + 1

In [9]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path,counter
0,Linear regression,Linear regression plays a fundamental role in ...,2012,https://www.cs.columbia.edu/~djhsu/coms4771-f2...,"[wcgZZP8AAAAJ, , El-UNYoAAAAJ]",Linear Regression,7cedd2b0-ac03-49fb-9ef1-398b1a5d7fe1.pdf,../../../detect_ml_model_files/7cedd2b0-ac03-4...,1
1,Introduction to linear regression analysis,Equation (1.2) is called a linear regression m...,2021,http://sutlib2.sut.ac.th/sut_contents/H133678.pdf,"[5PboKNAAAAAJ, , y2eb6cQAAAAJ]",Linear Regression,91374318-7135-4f51-a3bc-720ececd88de.pdf,../../../detect_ml_model_files/91374318-7135-4...,2
2,Linear regression,"linear regression, a very simple approach for ...",2023,https://datamineaz.org/readings/ISL_chp3.pdf,"[KUIjZqgAAAAJ, bHZf-c8AAAAJ, tQVe-fAAAAAJ, ZpG...",Linear Regression,6d3627b9-6b0b-423c-8898-3185837a7617.pdf,../../../detect_ml_model_files/6d3627b9-6b0b-4...,3
4,Applied linear regression,Applied linear regression Applied linear regre...,2005,https://www.stat.cmu.edu/~brian/valerie/617-20...,[wlu6jZQAAAAJ],Linear Regression,c7fed73c-8e13-4770-a589-67708aab0b7e.pdf,../../../detect_ml_model_files/c7fed73c-8e13-4...,4
8,A review on linear regression comprehensive in...,We discuss linear regression and polynomial re...,2020,https://jastt.org/index.php/jasttpath/article/...,"[G9U01kwAAAAJ, aBdgHxkAAAAJ]",Linear Regression,f8d408d7-9c86-4d72-b6af-6538ed46970f.pdf,../../../detect_ml_model_files/f8d408d7-9c86-4...,5
...,...,...,...,...,...,...,...,...,...
1227,BERT: a review of applications in natural lang...,"relative to the original BERT model, which is ...",2021,https://arxiv.org/pdf/2103.11943,[],BERT,91ebf298-65bd-4429-8c60-b17cb80306fe.pdf,../../../detect_ml_model_files/91ebf298-65bd-4...,1
1228,What does BERT learn about the structure of la...,language structure learned by BERT. We first s...,2019,https://inria.hal.science/hal-02131630/document,"[X7SMP1EAAAAJ, HXUT9ZkAAAAJ, P7EtARsAAAAJ]",BERT,65f04db1-ad62-4f48-9e80-3ae5a11bf147.pdf,../../../detect_ml_model_files/65f04db1-ad62-4...,2
1229,Visualizing and understanding the effectivenes...,trajectories of fine-tuning BERT on specific d...,2019,https://arxiv.org/pdf/1908.05620,"[cqOLO7IAAAAJ, wEfQgPgAAAAJ, G-V1VpwAAAAJ, ]",BERT,03d43387-8413-413c-8ce8-92b3512cedfb.pdf,../../../detect_ml_model_files/03d43387-8413-4...,3
1230,A comprehensive survey on pretrained foundatio...,Pretrained Foundation Models (PFMs) are regard...,2024,https://arxiv.org/pdf/2302.09419,"[HWx73DcAAAAJ, AHg-JGIAAAAJ, nPvWFpkAAAAJ, fh1...",BERT,a60911d8-b83b-4c2a-8555-2ac3b09b5025.pdf,../../../detect_ml_model_files/a60911d8-b83b-4...,4


In [10]:
# nlp = spacy.blank("en")
# ruler = nlp.add_pipe("entity_ruler")

#nlp = spacy.load("en_core_web_sm")
#ruler = nlp.add_pipe("entity_ruler", before="ner")

nlp = spacy.load("en_core_web_sm", exclude=["ner"])  # Exclude the default NER component
ruler = nlp.add_pipe("entity_ruler", last=True)

# Load the EntityRuler patterns from a file
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

Lets take the first four files for a model for training and the last one for test. This will give us a 80/20 split of files at least.

The query results from Google Scholar are in descending order of popularity. This should mean that the files are not necessarily in order of published date, which is good.

Further analysis is required to look at the terms in each file and create a more balanced training/test split.



In [11]:
training_files = data_df.query('counter < 5')['file_path'].tolist()

In [12]:
len(training_files)

396

In [13]:
test_files = data_df.query('counter == 5')['file_path'].tolist()

In [14]:
len(test_files)

99

There should be 4 times more training_files than test_files, lets test that

In [15]:
len(test_files) * 4 == len(training_files)

True

In [16]:
data_df.groupby(['query']).count().reset_index().query('counter != 5')['query'].values

array([], dtype=object)

In [17]:
# Test the pipeline
text = "Support Vector Machine, Support Vector Machines, SVM, and S.V.M. are popular machine learning methods."
doc = nlp(text)

# Print detected entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Support Vector Machine ML_METHOD
Support Vector Machines ML_METHOD
SVM ML_METHOD


Find terms in file to use as labels

In [18]:
def process_pdf_file(pdf_file_path):
    docs = []
    try:
        with open(pdf_file_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num, page in enumerate(pdf_reader.pages):
                # Extract text from the page
                text = page.extract_text()
                doc = nlp(text)
    
                # Extract sentences with ML_METHOD entities
                filtered_sentences = [
                    sentence.text
                    for sentence in doc.sents
                    if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
                ]
    
                if len(filtered_sentences) > 0:
                    # Create a new text containing only those sentences
                    filtered_text = " ".join(filtered_sentences)
    
                    # Create a new Doc object with the filtered text
                    filtered_doc = nlp(filtered_text)
                    docs.append(filtered_doc)
    except Exception as e:
        print(f"Error checking file {pdf_file_path}: {e}")
    
    return docs

def create_labelled_docs(files):
    all_docs = []
    with concurrent.futures.ProcessPoolExecutor() as executor:  # Use ProcessPoolExecutor for CPU-bound tasks
        results = list(executor.map(process_pdf_file, files))

    # Flatten the list of results
    for doc_list in results:
        all_docs.extend(doc_list)

    return all_docs

# Generate training data

In [19]:
training_docs = create_labelled_docs(files=training_files)

Error checking file ../../../detect_ml_model_files/c4cc64e6-dc8d-4972-ad07-887c977340d4.pdf: Missed the stop code in LZWDecode!


FloatObject (b'145.4927.471') invalid; use 0.0 instead
FloatObject (b'122.5144..673') invalid; use 0.0 instead
FloatObject (b'1418147.8804.673') invalid; use 0.0 instead
FloatObject (b'146.21128.543') invalid; use 0.0 instead
FloatObject (b'144.7421128.543') invalid; use 0.0 instead
FloatObject (b'125.9144.888') invalid; use 0.0 instead
FloatObject (b'15.631.489') invalid; use 0.0 instead
FloatObject (b'147.82.884') invalid; use 0.0 instead
FloatObject (b'141.4147.65') invalid; use 0.0 instead
FloatObject (b'147.28.043754') invalid; use 0.0 instead
FloatObject (b'6.6.745') invalid; use 0.0 instead
FloatObject (b'3147.319785.174') invalid; use 0.0 instead
FloatObject (b'14.319785.174') invalid; use 0.0 instead
FloatObject (b'147.488824.888') invalid; use 0.0 instead
FloatObject (b'130.7688837.248') invalid; use 0.0 instead
FloatObject (b'147.65.72941.99') invalid; use 0.0 instead
FloatObject (b'147.65.7793141.4147.65.77931416') invalid; use 0.0 instead
FloatObject (b'147.5345.77931416')

Error checking file ../../../detect_ml_model_files/488c7f95-6307-4142-8a05-efb5d3251af2.pdf: list index out of range


/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(
/home/reza/miniconda3/envs/ml-lncRNA/lib/python3.12/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H

In [22]:
if len(training_docs) < 1:
    print('There are no labels created for the requested files!!!')
else:
    # save it in a format spacy can use
    training_doc_bin = DocBin(docs=training_docs)
    #training_doc_bin.to_disk("./train.spacy")
    training_doc_bin.to_disk("./train_detect_ml_models.spacy")    

Lets visualise a random training doc

In [29]:
# Set a seed for repeatability
seed = 2048
random.seed(seed)

random_integer = random.randint(0, len(training_docs))

# Render the entities in a Jupyter Notebook or as HTML
options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(training_docs[random_integer], style="ent", options=options, jupyter=True)


# Generate test data

In [30]:
test_docs = create_labelled_docs(files=test_files)

Error checking file ../../../detect_ml_model_files/46647af5-8e95-41b1-b55b-8b2f20203c5b.pdf: EOF marker not found


XRef object at 4309 can not be read, some object may be missing
unknown widths : 
[0, IndirectObject(154, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(169, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(173, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(158, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(164, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(175, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(154, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(158, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(72, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(57, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(68, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(60, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(175, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(164, 0, 135103912798080)]
unknown widths : 
[0, IndirectObject(169, 0, 1351039127980

In [31]:
if len(test_docs) < 1:
    print('There are no labels created for the requested files!!!')
else:
    # save it in a format spacy can use
    test_doc_bin = DocBin(docs=test_docs)
    test_doc_bin.to_disk("./test_detect_ml_models.spacy")    

In [36]:
# Set a seed for repeatability
seed = 55
random.seed(seed)

random_integer = random.randint(0, len(test_docs))

# Render the entities in a Jupyter Notebook or as HTML
options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(test_docs[random_integer], style="ent", options=options, jupyter=True)